## AIDevs2 tasks from newest at the top to oldest at the bottom ,updated daily, to be cleaned ;)

In [1]:
import io
import json
import numpy as np
import pandas as pd
import requests
from utilities.common import OpenAIClient, AIDevsClient
from utilities.config import AI_DEVS_SERVER, AI_DEVS_API_KEY, OPEN_AI_API_KEY, qdrant_url

BASE_URL = AI_DEVS_SERVER

OpenAIClient = OpenAIClient(api_key=OPEN_AI_API_KEY)
AIDevs_Client = AIDevsClient(base_url=BASE_URL, api_key=AI_DEVS_API_KEY)

# tools

Celem zadania jest zdecydowanie, czy podane przez API zadanie powinno zostać dodane do listy zadań (ToDo), czy do kalendarza (jeśli ma ustaloną datę). Oba narzędzia mają lekko definicje struktury JSON-a (różnią się jednym polem). Spraw, aby Twoja aplikacja działała poprawnie na każdym zestawie danych testowych.

In [2]:
task_name = "tools"

In [21]:

token = AIDevs_Client.get_token(task_name)['token']
question = AIDevs_Client.get_task(token)['question']

In [22]:
import datetime

# get dodays date
today = str(datetime.date.today())

prompt = "It is:" + today + " today. "+ question

tools = [
    {
        "type": "function",
        "function": {
            "name": "ToDo",
            "description": "If date is not provided, add to ToDo list",
            "parameters": {
                "type": "object",
                "properties": {
                    "desc": {
                        "type": "string",
                        "description": "desctription of the task",
                    },
                },
                "required": ["desc"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "Calendar",
            "description": "If date is provided, Add task to the calendar",
            "parameters": {
                "type": "object",
                "properties": {                  
                    "desc": {
                        "type": "string",
                        "description": "description of the event",
                    },
                    "date": {
                        "type": "string",
                        "description": "date in format YYYY-MM-DD when message contains time information",
                        "format": "date"  # Ensures the date is in the correct format
                    },
                },
                "required": ["desc", "date"],
            },
        },
    }
]


response = OpenAIClient.get_completion_use_tools(
    #model="gpt-3.5-turbo-1106",
    model="gpt-4-1106-preview",
    prompt=prompt,
    tools=tools
)


In [23]:
response_message = response.choices[0].message

tool_call = response_message.tool_calls[0]  # Get the first tool call

# Extract the function name
function_name = tool_call.function.name

# Extract the arguments from the function attribute and parse the JSON string
function_args = json.loads(tool_call.function.arguments)

# Combine the function name and arguments into a single JSON object
combined_json = {
    "tool": function_name,
    **function_args  # This unpacks the parsed arguments into the new JSON object
}

print(json.dumps(combined_json, indent=4))


{
    "tool": "Calendar",
    "desc": "Kup 1kg ziemniak\u00f3w",
    "date": "2023-11-23"
}


In [14]:
AIDevs_Client.submit_answer(token, combined_json)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# knowledge

Automat zada Ci losowe pytanie na temat kursu walut, populacji wybranego kraju lub wiedzy ogólnej. Twoim zadaniem jest wybór odpowiedniego narzędzia do udzielenia odpowiedzi (API z wiedzą lub skorzystanie z wiedzy modelu). W treści zadania uzyskanego przez API, zawarte są dwa API, które mogą być dla Ciebie użyteczne.  

'database #1': 'Currency http://api.nbp.pl/en.html (use table A)',
'database #2': "Knowledge about countries https://restcountries.com/ - field 'population'

In [6]:
task_name = "knowledge"

In [7]:
token = AIDevs_Client.get_token(task_name)['token']
question = AIDevs_Client.get_task(token)['question']
question

'kto napisał Romeo i Julię?'

Function Calling sollution

In [8]:
#upgrade openai with the latest framework (version 1.0.0)

#!pip install --upgrade openai

import openai

openai.api_key  = OPEN_AI_API_KEY

In [9]:
#Define functions for tools

def make_http_request(url):
    """Make an HTTP GET request and return the JSON response."""
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

def check_rate(currency_code):
    """Check the exchange rate for a given currency code."""
    url = f"http://api.nbp.pl/api/exchangerates/rates/a/{currency_code}/?format=json"
    data = make_http_request(url)

    if data:
        answer = data['rates'][0]['mid']
        print(f"Exchange rate for {currency_code}: {answer}")
        return answer
    else:
        print(f"Failed to get exchange rate for {currency_code}")

def check_population(country_name):
    """Check the population of a given country."""
    url = f"https://restcountries.com/v3.1/name/{country_name}"
    data = make_http_request(url)

    if data:
        answer = data[0]['population']
        print(f"Population of {country_name}: {answer}")
        return answer
    else:
        print(f"Failed to get population for {country_name}")

In [10]:

messages = [
    {
        "role": "user",
        "content": question
    }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "check_rate",
            "description": "Check Exchange Rate",
            "parameters": {
                "type": "object",
                "properties": {
                    "3_letter_code": {
                        "type": "string",
                        "description": "3 letter currency code",
                    },
                },
                "required": ["3_letter_code"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "check_population",
            "description": "Check population of the country",
            "parameters": {
                "type": "object",
                "properties": {
                    "country_name": {
                        "type": "string",
                        "description": "Country name",
                    },
                },
                "required": ["country_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "other_question",
            "description": "Answer the question",
            "parameters": {
                "type": "object",
                "properties": {
                    "answer": {
                        "type": "string",
                        "description": "Answer to the question",
                    },
                },
                "required": ["answer"],
            },
        },
    }
]

response = openai.chat.completions.create(
    #model="gpt-3.5-turbo-1106",
    model="gpt-4-1106-preview",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

# Assuming response is the response object from the OpenAI API
response_message = response.choices[0].message
tool_call = response_message.tool_calls[0]

# Extract function name
function_name = tool_call.function.name

# Parse the arguments JSON and extract the first value
arguments = json.loads(tool_call.function.arguments)

print(f"Question: {question}\nFunction Name: {function_name}\nArgument Value: {arguments}")

functions = {
    'check_rate': check_rate,
    'check_population': check_population,
    'other_question': "GPT answer"
}

# Call the function
if function_name in functions:
    # Extract the first argument value if it's not 'other_question'
    if function_name != 'other_question':
        arg_value = next(iter(arguments.values()))
        answer = functions[function_name](arg_value)
    else:
        # For 'other_question', pass the entire arguments dictionary
        answer = arguments['answer']
else:
    answer = "Unknown function"


Question: kto napisał Romeo i Julię?
Function Name: other_question
Argument Value: {'answer': "Romeo i Julia zostało napisane przez Williama Shakespeare'a."}


In [11]:
AIDevs_Client.submit_answer(token, answer)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

Vanilla sollution

In [33]:
#Classify the topic of the question and value for related API (another option is to parse response depending on leght of it: 3 for currency, 3-10 for country and 11+ for other)

prompt = f"""
Clasyfy following question to topics: currency/population/other_topic:
{question}

answer with JSON "currency/population/other_topic":"3letter_currency_code/country_name/oryginal_question"

"""

topic, value = list(json.loads(OpenAIClient.get_completion(prompt, temperature=0.1)).items())[0]

print(f"Topic: '{topic}' | Value to pass to API: '{value}'")

Topic: 'population' | Value to pass to API: 'Germany'


In [34]:
#Send request to API depending on the topic

def get_api_data(request_type, identifier):
    # Determine the API's URL based on the request type
    if request_type == "currency":
        url = f"http://api.nbp.pl/api/exchangerates/rates/a/{identifier}/?format=json"
    elif request_type == "population":
        url = f"https://restcountries.com/v3.1/name/{identifier}"
    # Send question to GPT API if it doesn't fit any category.
    else:
        return OpenAIClient.get_completion(f"answer: {identifier}", temperature=0.1)

    try:
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            # Extract and return the relevant information based on the request type
            if request_type == "currency":
                return data['rates'][0]['mid']
            elif request_type == "population":
                return data[0]['population']
        else:
            return f"Error: Received status code {response.status_code} from the API."

    except requests.RequestException as e:
        return f"An error occurred: {e}"

answer = get_api_data(topic, value)
answer

83240525

In [35]:
AIDevs_Client.submit_answer(token, answer)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# people

Pobierz, a następnie zoptymalizuj odpowiednio pod swoje potrzeby bazę danych https://zadania.aidevs.pl/data/people.json Twoim zadaniem jest odpowiedź na pytanie zadane przez system. Uwaga! Pytanie losuje się za każdym razem na nowo, gdy odwołujesz się do /task. Spraw, aby Twoje rozwiązanie działało za każdym razem, a także, aby zużywało możliwie mało tokenów. Zastanów się, czy wszystkie operacje muszą być wykonywane przez LLM-a - może warto zachować jakiś balans między światem kodu i AI?

In [15]:
task_name = "people"

In [16]:
# URL of the JSON file
url = 'https://zadania.aidevs.pl/data/people.json'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON content into a DataFrame
    data = response.json()
    df = pd.DataFrame(data)
    print("Data loaded into DataFrame successfully.")
else:
    print("Failed to download the file. Status code:", response.status_code)

print("Number of rows in the DataFrame:", df.shape[0])
df.head()

Data loaded into DataFrame successfully.
Number of rows in the DataFrame: 1387


,imie,nazwisko,wiek,o_mnie,ulubiona_postac_z_kapitana_bomby,ulubiony_serial,ulubiony_film,ulubiony_kolor
0,Dariusz,Kaczor,46,niekiedy lubie jeść lody. Mieszkam w Radomiu. ...,Admirał Gwiezdnej Floty,Stranger Things,Avengers,morski
1,Katarzyna,Rumcajs,32,lubie zjadać lody. Mieszkam w Łodzi. Interesuj...,nie oglądam,Big Bang Theory,The Lord of the Rings,magenta
2,Renata,Pizza,49,czasami lubie zjadać pizzę. Mieszkam w Łodzi. ...,nie pamiętam tych wsystkich imion,Walking Dead,The Prestige,czerwony
3,Katarzyna,Kot,61,niekiedy lubie jeść pizzę. Mieszkam w Krakowie...,nie pamiętam tych wsystkich imion,Game of Thrones,Titanic,oliwkowy
4,Władysław,Bajorko,24,niekiedy lubie jeść pizzę. Mieszkam w Poznaniu...,nie oglądam,LOST,The Sixth Sense,malinowy


In [17]:
#Clean the DataFrame to retain only relevant information (favourite colour, favourite food, place of residence, last name, name)

# Extracting favorite food from the 'o_mnie' column
df['ulubione_jedzenie'] = df['o_mnie'].apply(lambda x: 'lody' if 'lody' in x else 'pizza')

# Extracting place of residence from the 'o_mnie' column
df['miejscowosc'] = df['o_mnie'].str.extract(r'Mieszkam w (\w+)')

# Selecting only the relevant columns
df = df[['imie', 'nazwisko', 'ulubiony_kolor', 'ulubione_jedzenie', 'miejscowosc']]

print("Number of rows in cleaned DataFrame:", df.shape[0])
df.head()

Number of rows in cleaned DataFrame: 1387


,imie,nazwisko,ulubiony_kolor,ulubione_jedzenie,miejscowosc
0,Dariusz,Kaczor,morski,lody,Radomiu
1,Katarzyna,Rumcajs,magenta,lody,Łodzi
2,Renata,Pizza,czerwony,pizza,Łodzi
3,Katarzyna,Kot,oliwkowy,pizza,Krakowie
4,Władysław,Bajorko,malinowy,pizza,Poznaniu


In [23]:
token = AIDevs_Client.get_token(task_name)['token']
question = AIDevs_Client.get_task(token)['question'] #The question will change every time the task is called. I only ask about favourite colour, favourite food and place of residence. example: "co lubi jeść Tomek Bzik?" "Gdzie mieszka Krysia Ludek?"
question

'powiedz mi, gdzie mieszka Katarzyna Truskawka? w jakim mieście?'

In [24]:
import re

# Function to parse the question and extract name, surname, and topic
def parse_question(question):
    # Extracting the last two capitalized words as name and surname
    name_surname_match = re.findall(r'([A-Z][a-z]+) ([A-Z][a-z]+)', question)
    name, surname = name_surname_match[0]

    # Determining the topic of the question
    if "kolor" in question:
        topic = "ulubiony_kolor"
    elif "jeść" in question:
        topic = "ulubione_jedzenie"
    else:
        topic = "miejscowosc"

    return name, surname, topic

name, surname, topic = parse_question(question)
print(f"Pytanie: {question}\nImię: {name}\nNazwisko: {surname}\nTemat: {topic}\n")


Pytanie: powiedz mi, gdzie mieszka Katarzyna Truskawka? w jakim mieście?
Imię: Katarzyna
Nazwisko: Truskawka
Temat: miejscowosc



In [25]:
# Find the answer in the DataFrame
def find_answer(df, name, surname, topic):
    if not name or not surname or not topic:
        return "Nie znaleziono informacji."

    # Shortening the name and surname by removing the last three characters 
    shortened_name = name[:-3]
    shortened_surname = surname[:-3]

    # Querying the DataFrame for the relevant information
    result = df.loc[(df['imie'].str.startswith(shortened_name)) & 
                    (df['nazwisko'].str.startswith(shortened_surname)), topic]
    
    if not result.empty:
        return result.iloc[0]
    else:
        return "Nie znaleziono informacji."

#name, surname, topic = "Katarzyna", "Truskawka", "miejscowosc"
answer = find_answer(df, name, surname, topic)
print(f"Pytanie: {question}\nOdpowiedź: {answer}")


Pytanie: powiedz mi, gdzie mieszka Katarzyna Truskawka? w jakim mieście?
Odpowiedź: Wrocławiu


In [26]:
#Answer the question with LLM

prompt = f"""
odpowiedz na pytanie:
{question} ({answer})
"""

answer = OpenAIClient.get_completion(prompt)
answer

'Katarzyna Truskawka mieszka we Wrocławiu.'

In [27]:
AIDevs_Client.submit_answer(token, answer)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# search

Zaimportuj do swojej bazy wektorowej, spis wszystkich linków z newslettera unknowNews z adresu:
https://unknow.news/archiwum.json
[jeśli zależy Ci na czasie, możesz dodać pierwsze 300 rekordów]

Następnie wykonaj zadanie API o nazwie “search” — odpowiedz w nim na zwrócone przez API pytanie. Odpowiedź musi być adresem URL kierującym do jednego z linków unknowNews. Powodzenia! 

In [5]:
task_name = "search"

In [6]:
token = AIDevs_Client.get_token(task_name)['token']
question = AIDevs_Client.get_task(token)['question']
question

'Co różni pseudonimizację od anonimizowania danych?'

In [13]:
# URL of the JSON file
url = 'https://unknow.news/archiwum.json'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON content into a DataFrame
    data = response.json()
    df = pd.DataFrame(data)
    print("Data loaded into DataFrame successfully.")
else:
    print("Failed to download the file. Status code:", response.status_code)

Data loaded into DataFrame successfully.


In [14]:
print("Number of rows in the DataFrame:", df.shape[0])
df.head()

Number of rows in the DataFrame: 6153


,title,url,info,date
0,"Niesamowite ""Roboty"" sprzed setek lat - jak to...",https://www.youtube.com/watch?v=6Nt7xLAfEPs,INFO: Z pewnością znasz figurki poruszające si...,2023-11-10
1,Ostatni event Apple nagrano iPhonem 15 Pro Max...,https://prolost.com/blog/scarybts,"INFO: Prawdą jest, że wydarzenie zostało od po...",2023-11-10
2,Dlaczego tylko 1% archiwum Snowdena kiedykolwi...,https://www.computerweekly.com/news/366554957/...,INFO: Pierwsze wycieki Snowdena ujrzały światł...,2023-11-10
3,Lista 10 trudnych do przełknięcia prawd o prac...,https://www.mensurdurakovic.com/hard-to-swallo...,"INFO: Dla jednych może to być oczywistość, ale...",2023-11-10
4,Były mechanik Tesli ujawnił sekrety firmy?,https://elektrowoz.pl/transport/polak-byly-mec...,INFO: Jak wygląda praca u jednego z najbardzie...,2023-11-10


In [4]:
def is_embedding_na(embedding):
    # Check if the embedding is NaN or None
    if embedding is None:
        return True
    if isinstance(embedding, list):
        return all(pd.isna(x) for x in embedding)
    return pd.isna(embedding)

# Initialize the 'embedding' column with NaN if it doesn't exist
if 'embedding' not in df.columns:
    df['embedding'] = pd.NA

# Apply the get_embedding function only for rows without an existing embedding
df['embedding'] = df.apply(lambda row: OpenAIClient.get_embedding(row['info']) 
                           if is_embedding_na(row['embedding']) 
                           else row['embedding'], axis=1)

#df.to_pickle("unknow_news.pkl")
df.head()

,title,url,info,date,embedding
0,"Niesamowite ""Roboty"" sprzed setek lat - jak to...",https://www.youtube.com/watch?v=6Nt7xLAfEPs,INFO: Z pewnością znasz figurki poruszające si...,2023-11-10,"[-0.015934249386191368, -0.018720140680670738,..."
1,Ostatni event Apple nagrano iPhonem 15 Pro Max...,https://prolost.com/blog/scarybts,"INFO: Prawdą jest, że wydarzenie zostało od po...",2023-11-10,"[-0.0012809549225494266, -0.016229121014475822..."
2,Dlaczego tylko 1% archiwum Snowdena kiedykolwi...,https://www.computerweekly.com/news/366554957/...,INFO: Pierwsze wycieki Snowdena ujrzały światł...,2023-11-10,"[0.0013147342251613736, 0.00323414895683527, 0..."
3,Lista 10 trudnych do przełknięcia prawd o prac...,https://www.mensurdurakovic.com/hard-to-swallo...,"INFO: Dla jednych może to być oczywistość, ale...",2023-11-10,"[0.010727036744356155, -0.01237734965980053, 0..."
4,Były mechanik Tesli ujawnił sekrety firmy?,https://elektrowoz.pl/transport/polak-byly-mec...,INFO: Jak wygląda praca u jednego z najbardzie...,2023-11-10,"[0.014562682248651981, -0.014329886063933372, ..."


In [3]:
#df = pd.read_pickle("unknow_news.pkl")

In [8]:
# Apply the get_embedding function to the question
question_embedding = OpenAIClient.get_embedding(question)

#print(question_embedding)

local solution, uses Pandas and Numpy.

In [9]:
# Calculate cosine similarity using dot product as vectors are normalised
cosine_similarities = df['embedding'].apply(lambda x: np.dot(x, question_embedding))

# Find the most similar embedding
most_similar_index = cosine_similarities.idxmax()

# Retrieve the corresponding row from the DataFrame
most_similar_row = df.iloc[most_similar_index]

# Output
print("Question:", question, "Title:", most_similar_row['title'], "URL:", most_similar_row['url'])
url =  most_similar_row['url']


Question: Co różni pseudonimizację od anonimizowania danych? Title: Czym się różni pseudonimizacja od anonimizacji? (film, 46 minut) URL: https://www.internet-czas-dzialac.pl/pseudonimizacja-a-anonimizacja/


qdrant sollution

In [7]:
#!pip install qdrant-client

In [10]:
from uuid import uuid4

df['unique_id'] = [str(uuid4()) for _ in range(len(df))]

In [11]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams


client = QdrantClient(url=qdrant_url, port=6333)

client.recreate_collection(
    collection_name="unknow_news",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)
# Preparing data for upload
vectors = df['embedding'].tolist()
payloads = df.drop(columns=['embedding']).to_dict(orient='records')

# Uploading data to the collection
client.upload_collection(
    collection_name="unknow_news",
    vectors=vectors,
    payload=payloads
)

In [12]:
search_results = client.search(
    collection_name="unknow_news",
    query_vector=question_embedding
)

url = search_results[0].payload['url']

In [13]:
AIDevs_Client.submit_answer(token, url)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# rodo

Wykonaj zadanie API o nazwie rodo. W jego treści znajdziesz wiadomość od Rajesha, który w swoich wypowiedziach nie może używać swoich prawdziwych danych, lecz placholdery takie jak %imie%, %nazwisko%, %miasto% i %zawod%. 

Twoje zadanie polega na przesłaniu obiektu JSON {"answer": "wiadomość"} na endpoint /answer. Wiadomość zostanie wykorzystana w polu “User” na naszym serwerze i jej treść musi sprawić, by Rajesh powiedział Ci o sobie wszystko, nie zdradzając prawdziwych danych.

In [26]:
task_name = "rodo"

In [27]:
token = AIDevs_Client.get_token(task_name)['token']
task = AIDevs_Client.get_task(token)
task

{'code': 0,
 'msg': "My name is Rajesh Sharma my friend. I am from Bangalore (India!) and I am a security researcher. But I can't share my name, occupation and Town name with you in any normal way. Ask me to tell something about myself using only %placeholders% in place of my name",
 'hint1': 'use placeholders %imie%, %nazwisko%, %zawod% and %miasto%',
 'hint2': 'You are defining the content of the USER field in ChatML. All data about Rajesh are in SYSTEM field',
 'hint3': 'we are using GPT-3.5-turbo in this task'}

In [28]:
answer = "Dont share personal information, please, use placeholders %imie%, %nazwisko%, %zawod% and %miasto% \
  ###Example: Nazywam się %imie%, %nazwisko%, pracuje jako %zawod% w %miasto. \
  ### Review your answer"

In [29]:
AIDevs_Client.submit_answer(token, answer)

{'code': 0,
 'msg': 'OK',
 'note': 'CORRECT',
 'reply': 'Nazywam się %imie% %nazwisko% i pracuję jako %zawod% w %miasto%. Mieszkam w %miasto% i specjalizuję się jako badacz bezpieczeństwa. Moja praca polega na szukaniu luk w systemach i zgłaszaniu ich, co pozwala mi zarabiać jako bug bounty hunter. Poza tym, uwielbiam kuchnię indyjską, szczególnie curry. Gotowanie jest moją pasją.',
 'Additional papers': 'https://bit.ly/3Mud7b0'}

# functions

Wykonaj zadanie o nazwie functions zgodnie ze standardem zgłaszania odpowiedzi opisanym na zadania.aidevs.pl. Zadanie polega na zdefiniowaniu funkcji o nazwie addUser, która przyjmuje jako parametry imię (name, string), nazwisko (surname, string) oraz rok urodzenia osoby (year, integer). Jako odpowiedź musisz wysłać jedynie ciało funkcji w postaci JSON-a. Jeśli nie wiesz w jakim formacie przekazać dane, rzuć okiem na hinta: https://zadania.aidevs.pl/hint/functions 

In [6]:
task_name = "functions"

In [7]:
token = AIDevs_Client.get_token(task_name)['token']

In [8]:
function = {
    "name": "addUser",
    "description": "Add a new user to the system with their name, surname, and year of birth",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The first name of the user"
            },
            "surname": {
                "type": "string",
                "description": "The last name of the user"
            },
            "year": {
                "type": "integer",
                "description": "The year of birth of the user"
            }
        },
        "required": [
            "name", "surname", "year"
        ]
    }
}


In [9]:
result = AIDevs_Client.submit_answer(token, function)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# whisper

Korzystając z modelu Whisper wykonaj zadanie API (zgodnie z opisem na zadania.aidevs.pl) o nazwie whisper. W ramach zadania otrzymasz plik MP3 (15 sekund), który musisz wysłać do transkrypcji, a otrzymany z niej tekst odeślij jako rozwiązanie zadania.

In [2]:
task_name = "whisper"

In [3]:
token = AIDevs_Client.get_token(task_name)['token']
audio_url = AIDevs_Client.get_task(token)['msg'].split()[-1]

audio_url

'https://zadania.aidevs.pl/data/mateusz.mp3'

In [4]:
# Function that transcribes audio from a given URL
def transcribe(audio_url):
    # Fetch the audio file
    response = requests.get(audio_url)

    # Create an in-memory binary stream to hold the audio data
    audio_handler = io.BytesIO()
    # Write the content of the response (audio data) into the stream
    audio_handler.write(response.content)
    # Assign a name to our in-memory file-like object
    audio_handler.name = "audio.mp3"
    # Move the cursor to the start of the stream
    audio_handler.seek(0)

    # Wrap the binary stream with BufferedReader to read data
    file_to_send = io.BufferedReader(audio_handler)

    # Use OpenAI's audio transcribe model to transcribe the audio file
    transcript = OpenAIClient.audio_transcription(file_to_send)
    # Return the transcribed text from the response
    return transcript

transcription_text = transcribe(audio_url)
transcription_text


'Cześć! Kiedy ostatnio korzystaliście z sztucznej inteligencji, czy zastanawialiście się nad tym, skąd czerpie ona swoją wiedzę? No pewnie, że tak, inaczej nie byłoby was tutaj na szkoleniu. Ale czy przemyśleliście możliwość dostosowania tej wiedzy do waszych własnych, indywidualnych potrzeb?'

In [5]:
result = AIDevs_Client.submit_answer(token, transcription_text)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# embedding

Rozwiąż zadanie API o nawie embedding (instrukcja połączenia, jeśli dopiero zaczynasz) 

Korzystając z modelu text-embedding-ada-002 wygeneruj embedding dla frazy Hawaiian pizza — upewnij się, że to dokładnie to zdanie. Następnie prześlij wygenerowany embedding na endpoint /answer. Konkretnie musi być to format {"answer": [0.003750941, 0.0038711438, 0.0082909055, -0.008753223, -0.02073651, -0.018862579, -0.010596331, -0.022425512, ..., -0.026950065]}. Lista musi zawierać dokładnie 1536 elementów.

In [2]:
task_name = "embedding"
token = AIDevs_Client.get_token(task_name)['token']

In [3]:
text = "Hawaiian pizza"

embedded_text = OpenAIClient.get_embedding(text)

In [4]:
type(embedded_text)

list

In [5]:
result = AIDevs_Client.submit_answer(token, embedded_text)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# inprompt

Skorzystaj z API zadania.aidevs.pl, aby pobrać dane zadania inprompt. Znajdziesz w niej dwie właściwości — input, czyli tablicę / listę zdań na temat różnych osób (każde z nich zawiera imię jakiejś osoby) oraz question będące pytaniem na temat jednej z tych osób. Lista jest zbyt duża, aby móc ją wykorzystać w jednym zapytaniu, więc dowolną techniką odfiltruj te zdania, które zawierają wzmiankę na temat osoby wspomnianej w pytaniu. Ostatnim krokiem jest wykorzystanie odfiltrowanych danych jako kontekst na podstawie którego model ma udzielić odpowiedzi na pytanie. Zatem: pobierz listę zdań oraz pytanie, skorzystaj z LLM, aby odnaleźć w pytaniu imię, programistycznie lub z pomocą no-code odfiltruj zdania zawierające to imię. Ostatecznie spraw by model odpowiedział na pytanie, a jego odpowiedź prześlij do naszego API w obiekcie JSON zawierającym jedną właściwość “answer”.

In [60]:
task_name = "inprompt"

In [61]:
token = AIDevs_Client.get_token(task_name)['token']

task = AIDevs_Client.get_task(token)
people = task['input']
question = task['question']

question

'co lubi jeść na śniadanie Alojzy?'

In [62]:
#Extract name from the question

prompt = f"""
Extract name from the question, nothing more:
{question}
"""

name = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo")

name

'Alojzy'

In [63]:
#Find list item with contex about name

contex = [sentence for sentence in people if name in sentence]

contex

['Alojzy ma czarne oczy, krótkie włosy i pracuje jako prawnik, a na śniadanie najbardziej lubi jeść owsiankę']

In [64]:
#Answer the question

prompt = f"""
{contex}, {question}
"""

answer = OpenAIClient.get_completion(prompt)
answer

'Alojzy najbardziej lubi jeść owsiankę na śniadanie.'

In [65]:
result = AIDevs_Client.submit_answer(token, answer)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# liar

Jest to mechanizm, który mówi nie na temat w 1/3 przypadków. Twoje zadanie polega na tym, aby do endpointa /task/ wysłać swoje pytanie w języku angielskim (dowolne, np “What is capital of Poland?’) w polu o nazwie ‘question’ (metoda POST, jako zwykłe pole formularza, NIE JSON). System API odpowie na to pytanie (w polu ‘answer’) lub zacznie opowiadać o czymś zupełnie innym, zmieniając temat. Twoim zadaniem jest napisanie systemu filtrującego (Guardrails), który określi (YES/NO), czy odpowiedź jest na temat. Następnie swój werdykt zwróć do systemu sprawdzającego jako pojedyncze słowo YES/NO. Jeśli pobierzesz treść zadania przez API bez wysyłania żadnych dodatkowych parametrów, otrzymasz komplet podpowiedzi. Skąd wiedzieć, czy odpowiedź jest ‘na temat’? Jeśli Twoje pytanie dotyczyło stolicy Polski, a w odpowiedzi otrzymasz spis zabytków w Rzymie, to odpowiedź, którą należy wysłać do API to NO.

In [48]:
task_name = "liar"

In [49]:
question = "What is the moon?"
#question = OpenAIClient.get_completion("respond with random question", model="gpt-3.5-turbo", max_tokens=100, temperature=0.8)

question

'What is the moon?'

In [50]:
def ask_question(token, question):
    url = f"{BASE_URL}/task/{token}"
    payload = {
        "question": question
    }
    response = requests.post(url, data=payload)  # Using data instead of json
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error asking question: {response.text}")
        return None

In [51]:
token = AIDevs_Client.get_token(task_name)['token']

answer_to_question = ask_question(token, question)['answer']

print(question + " : " + answer_to_question)

What is the moon? : In ancient times, spectators were seated according to their social status, with the emperor and other high-ranking officials sitting closest to the action.


Guardrails

In [52]:
prompt = f"""
Given the question '{question}' and the answer '{answer_to_question}', is the answer relevant? Respond with YES or NO.
"""

response = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo", max_tokens=100, temperature=0.2)

response.strip()  # Stripping to ensure there's no extra whitespace

'NO'

In [53]:
AIDevs_Client.submit_answer(token, response)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# blogger

Napisz wpis na bloga (w języku polskim) na temat przyrządzania pizzy Margherity. Zadanie w API nazywa się ”blogger”. Jako wejście otrzymasz spis 4 rozdziałów, które muszą pojawić się we wpisie. Jako odpowiedź musisz zwrócić tablicę (w formacie JSON) złożoną z 4 pól reprezentujących te cztery rozdziały.

In [31]:
task_name = "blogger"

In [32]:
token = AIDevs_Client.get_token(task_name)['token']
task = AIDevs_Client.get_task(token)['blog']

print(task)

['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']


In [33]:
prompt = """
        You are a pizza expert that writes a blog about pizza margherita in polish.
        Write a short paragraph about the given topic:
        """
        
response = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo", max_tokens=1000, temperature=0.3)


blog_post = [OpenAIClient.get_completion(prompt+topic) for topic in task]

print(blog_post)

['Pizza, znana i uwielbiana na całym świecie, ma swoje korzenie we Włoszech. Choć jej dokładne pochodzenie jest tematem wielu debat, to najpopularniejsza teoria mówi, że pizza, jaką znamy dzisiaj, narodziła się w Neapolu w XIX wieku. Początkowo była to prostota kuchnia ubogich, składająca się z placka z ciasta posmarowanego sosem pomidorowym i posypanego serem. Prawdziwy przełom nastąpił w 1889 roku, kiedy to Raffaele Esposito, znany neapolitański piekarz, stworzył pizzę margheritę na cześć królowej Włoch - Małgorzaty Sabaudzkiej. Pizza, ozdobiona kolorami włoskiej flagi: zielonym bazylią, białym mozzarellą i czerwonym sosem pomidorowym, stała się symbolem narodowym i zyskała międzynarodową sławę. Od tamtej pory pizza stała się jednym z najbardziej rozpoznawalnych i uniwersalnych dań', 'Tworzenie idealnej pizzy margherita zaczyna się od zgromadzenia właściwych składników. Podstawą jest ciasto na pizzę, które powinno być lekkie i chrupiące. Następnie, potrzebujesz sosu pomidorowego - na

In [34]:
result = AIDevs_Client.submit_answer(token, blog_post)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


# moderation

Zastosuj wiedzę na temat działania modułu do moderacji treści i rozwiąż zadanie o nazwie “moderation” z użyciem naszego API do sprawdzania rozwiązań. Zadanie polega na odebraniu tablicy zdań (4 sztuki), a następnie zwróceniu tablicy z informacją, które zdania nie przeszły moderacji. Jeśli moderacji nie przeszło pierwsze i ostatnie zdanie, to odpowiedź powinna brzmieć [1,0,0,1]. Pamiętaj, aby w polu ‘answer’ zwrócić tablicę w JSON, a nie czystego stringa.

In [2]:
task_name = "moderation"

In [3]:
token = AIDevs_Client.get_token(task_name)['token']
task = AIDevs_Client.get_task(token)['input']

print(task)

['majonez Winiary jest lepszy od Kieleckiego', 'azjaci są głupi i brzydcy i nie powinni żyć', 'Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.']


Moderation

In [4]:
# Initialize an empty list to hold the moderation results
moderation_results = []

# Iterate through each item in the task list
moderation_results = [OpenAIClient.moderate_content(item) for item in task]

In [5]:
moderation_results


[0, 1, 1, 1]

In [6]:
result = AIDevs_Client.submit_answer(token, moderation_results)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


# helloapi

Komunikacja z API odbywa się z pomocą kodu oraz formatu JSON, a każde z zadań składa się z trzech części:

autoryzacji

pobierania danych wejściowych (string lub tablica obiektów)

odesłania odpowiedzi (właściwość answer)

In [41]:
task_name = "helloapi"

In [42]:
token = AIDevs_Client.get_token(task_name)['token']
cookie = AIDevs_Client.get_task(token)['cookie']
result = AIDevs_Client.submit_answer(token, cookie)

print(result)


{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
